# NCAA Score Prediction (Linear Regression)
## Tony Ghabour
***

## Data Collection/Scraping/Transformation

In [1]:
import sys
sys.path.append('../src')

In [2]:
#Initial setup

import pandas as pd
import requests
from bs4 import BeautifulSoup
import NCAA_functions as ncaa

In [3]:
# Create DataFrame of all NCAA schools. Later will add columns for teams' average 
# heights and weights as features for linear regression.

all_schools = ncaa.get_schools() 
all_schools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   School       503 non-null    object
 1   City, State  503 non-null    object
dtypes: object(2)
memory usage: 8.0+ KB


In [4]:
all_schools.head()

,School,"City, State"
0,Abilene Christian Wildcats,"Abilene, Texas"
1,Air Force Falcons,"USAF Academy, Colorado"
2,Akron Zips,"Akron, Ohio"
3,Alabama A&M Bulldogs,"Normal, Alabama"
4,Alabama Crimson Tide,"Tuscaloosa, Alabama"


In [5]:
# Set up objects we'll need for scraping roster information.

url = 'https://www.sports-reference.com/cbb/schools/'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "html5lib")

In [6]:
# Create dictionary with school names as keys and partial urls to school page as values.

school_links = {}
for link in soup.find_all('a'):
    if link['href'][:13] == '/cbb/schools/':
        sch = link.text
        link = link['href'][13:]
        school_links[sch] = link

In [7]:
# Populate DataFrame with partial urls for each school. 

for index, row in all_schools.iterrows():
    try:
        all_schools.loc[index, 'Link'] = school_links[row.School]
    except:
        pass

In [8]:
# Eliminate broken/meaningless urls (e.g. when table has breaks, column headers) 
# and confirm new column populated properly.

all_schools = all_schools.dropna()
all_schools.head()

,School,"City, State",Link
0,Abilene Christian Wildcats,"Abilene, Texas",abilene-christian/
1,Air Force Falcons,"USAF Academy, Colorado",air-force/
2,Akron Zips,"Akron, Ohio",akron/
3,Alabama A&M Bulldogs,"Normal, Alabama",alabama-am/
4,Alabama Crimson Tide,"Tuscaloosa, Alabama",alabama/


In [9]:
counter = 0

for index, row in all_schools.iterrows():
    
    url = 'https://www.sports-reference.com/cbb/schools/' + row['Link']

    weight_2016, height_2016 = ncaa.scrape_single_roster(url, 2016)
    weight_2017, height_2017 = ncaa.scrape_single_roster(url, 2017)

    try:
        all_schools.loc[index, '2016_weight'] = weight_2016
        all_schools.loc[index, '2016_height'] = height_2016

        all_schools.loc[index, '2017_weight'] = weight_2017
        all_schools.loc[index, '2017_height'] = height_2017
    except:
        pass
    
    counter += 1
    
    print(str(counter)+ '...' + row['School'] + '...DONE')   

1...Abilene Christian Wildcats...DONE
2...Air Force Falcons...DONE
3...Akron Zips...DONE
4...Alabama A&M Bulldogs...DONE
5...Alabama Crimson Tide...DONE
6...Alabama State Hornets...DONE
7...Alabama-Birmingham Blazers...DONE
8...Albany (NY) Great Danes...DONE
9...Alcorn State Braves...DONE
10...Allegheny Gators...DONE
11...American Eagles...DONE
12...Amherst Lord Jeffs...DONE
13...Appalachian State Mountaineers...DONE
14...Arizona State Sun Devils...DONE
15...Arizona Wildcats...DONE
16...Arkansas Razorbacks...DONE
17...Arkansas State Red Wolves...DONE
18...Arkansas-Pine Bluff Golden Lions...DONE
19...Armstrong Pirates...DONE
20...Army Black Knights...DONE
21...Auburn Tigers...DONE
22...Augusta State Jaguars...DONE
23...Augustana (IL) Vikings...DONE
24...Austin Peay Governors...DONE
25...Baker University Wildcats...DONE
26...Baldwin-Wallace Yellow Jackets...DONE
27...Ball State Cardinals...DONE
28...Baltimore Super Bees...DONE
29...Baylor Bears...DONE
30...Belmont Bruins...DONE
31...Belo

240...Missouri State Bears...DONE
241...Missouri Tigers...DONE
242...Missouri-Kansas City Kangaroos...DONE
243...Monmouth Hawks...DONE
244...Montana Grizzlies...DONE
245...Montana State Bobcats...DONE
246...Morehead State Eagles...DONE
247...Morgan State Bears...DONE
248...Morris Brown Wolverines...DONE
249...Mount St. Mary's Mountaineers...DONE
250...Mount Union Purple Raiders...DONE
251...Muhlenburg Mules...DONE
252...Murray State Racers...DONE
253...Muskingum Fighting Muskies...DONE
254...Navy Midshipmen...DONE
255...Nebraska Cornhuskers...DONE
256...Nebraska Wesleyan Prairie Wolves...DONE
257...Nevada Wolf Pack...DONE
258...Nevada-Las Vegas Rebels...DONE
259...New Hampshire Wildcats...DONE
260...New Mexico Lobos...DONE
261...New Mexico State Aggies...DONE
262...New Orleans Privateers...DONE
263...New York University Violets...DONE
264...Newberry Wolves...DONE
265...Niagara Purple Eagles...DONE
266...Nicholls State Colonels...DONE
267...NJIT Highlanders...DONE
268...Norfolk State Sp

470...Xavier Musketeers...DONE
471...Yale Bulldogs...DONE
472...Youngstown State Penguins...DONE


In [10]:
# Drop schools for which no height/weight information is available.   
# These are generally schools that no longer sponsor D1 NCAA basketball teams. 

all_schools = all_schools.dropna()
all_schools.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 502
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   School       351 non-null    object 
 1   City, State  351 non-null    object 
 2   Link         351 non-null    object 
 3   2016_weight  351 non-null    float64
 4   2016_height  351 non-null    float64
 5   2017_weight  351 non-null    float64
 6   2017_height  351 non-null    float64
dtypes: float64(4), object(3)
memory usage: 21.9+ KB


In [11]:
all_schools.head()

,School,"City, State",Link,2016_weight,2016_height,2017_weight,2017_height
0,Abilene Christian Wildcats,"Abilene, Texas",abilene-christian/,202.272727,75.818182,202.307692,76.000000
1,Air Force Falcons,"USAF Academy, Colorado",air-force/,196.850000,76.650000,196.736842,76.842105
2,Akron Zips,"Akron, Ohio",akron/,208.857143,77.285714,200.933333,75.800000
3,Alabama A&M Bulldogs,"Normal, Alabama",alabama-am/,195.526316,75.842105,196.785714,74.571429
4,Alabama Crimson Tide,"Tuscaloosa, Alabama",alabama/,208.500000,77.250000,211.285714,77.857143


In [14]:
# Export scraped data to file

all_schools.to_csv(r'../data/raw/NCAA_scraped_data.csv', index = True, header=True) 

Includes game-level statistics for both home and away teams.   
Only considering Men's D1, non-neutral site games from 2016-2017 and 2017-2018 seasons.

In [15]:
# Downloaded from: 
# https://console.cloud.google.com/marketplace/details/ncaa-bb-public/ncaa-basketball

game_data = ncaa.get_game_data()
game_data.info()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (4,7,16,21) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9518 entries, 6469 to 29785
Data columns (total 70 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   game_id                   9518 non-null   object 
 1   season                    9518 non-null   object 
 2   scheduled_date            9518 non-null   object 
 3   lead_changes              9465 non-null   float64
 4   times_tied                9462 non-null   float64
 5   periods                   9518 non-null   float64
 6   h_name                    9518 non-null   object 
 7   h_alias                   9518 non-null   object 
 8   h_points_game             9518 non-null   int64  
 9   h_field_goals_made        9518 non-null   float64
 10  h_field_goals_att         9518 non-null   float64
 11  h_field_goals_pct         9518 non-null   float64
 12  h_three_points_made       9518 non-null   float64
 13  h_three_points_att        9518 non-null   float64
 14  h_th

In [16]:
game_data.head()

,game_id,season,scheduled_date,lead_changes,times_tied,periods,h_name,h_alias,h_points_game,h_field_goals_made,...,a_blocks,a_assists_turnover_ratio,a_personal_fouls,a_foulouts,a_points,a_fast_break_pts,a_second_chance_pts,a_team_turnovers,a_points_off_turnovers,a_team_rebounds
6469,8c8d2a48-2335-4acd-8304-8658b7c4f193,2017,2018-03-02,6.0,9.0,2.0,Huskies,WASH,79,24.0,...,2.0,1.00,21.0,1.0,77.0,0.0,16.0,2.0,10.0,1.0
6470,0910b42c-d65f-44da-8076-f1bfcf2b3399,2017,2018-03-02,0.0,0.0,2.0,Sun Devils,ASU,84,26.0,...,3.0,0.35,22.0,1.0,53.0,8.0,10.0,0.0,8.0,7.0
6471,b0cf1cc3-85c3-4a81-84a9-5c45624e0dfa,2017,2018-03-02,9.0,5.0,2.0,Cougars,WSU,78,28.0,...,3.0,1.08,20.0,0.0,76.0,6.0,9.0,0.0,14.0,1.0
6472,4eb7e921-ecf2-4c3c-af68-590d44b9c717,2017,2018-03-02,2.0,3.0,2.0,Wildcats,ARIZ,75,28.0,...,0.0,1.71,20.0,0.0,67.0,4.0,8.0,1.0,18.0,3.0
6473,9fd12195-25c0-4a97-8ea9-c2308b538d6b,2017,2018-03-04,10.0,3.0,2.0,Trojans,USC,72,26.0,...,6.0,1.60,20.0,0.0,83.0,15.0,11.0,0.0,6.0,1.0


In [17]:
# Inspect unique values in each field and confirm there are no obviously erroneous entries.

for idx, column in enumerate(game_data.columns):
        print(game_data.columns[idx], game_data[column].unique())

game_id ['8c8d2a48-2335-4acd-8304-8658b7c4f193'
 '0910b42c-d65f-44da-8076-f1bfcf2b3399'
 'b0cf1cc3-85c3-4a81-84a9-5c45624e0dfa' ...
 '4c584e1f-0d8b-4f27-bea4-534d08f9021e'
 '4a08564f-0f9b-447c-ae71-176c96871269'
 '8cd232f7-2bbc-4812-97e8-479e08727e47']
season ['2017' '2016']
scheduled_date ['2018-03-02' '2018-03-04' '2018-03-03' '2017-12-17' '2017-02-03'
 '2017-12-16' '2016-12-31' '2017-11-15' '2017-11-13' '2017-01-02'
 '2017-02-04' '2017-02-18' '2016-11-18' '2017-11-30' '2018-02-03'
 '2017-02-25' '2018-01-05' '2017-01-15' '2018-01-08' '2016-12-01'
 '2017-12-31' '2018-01-18' '2017-01-13' '2018-02-25' '2017-11-17'
 '2016-12-23' '2018-02-02' '2017-02-23' '2016-11-16' '2018-01-20'
 '2018-02-23' '2016-11-20' '2017-12-20' '2018-01-12' '2017-01-09'
 '2017-11-18' '2017-02-10' '2016-11-14' '2017-02-19' '2016-12-15'
 '2018-01-01' '2018-02-18' '2016-12-22' '2018-01-26' '2017-12-30'
 '2017-03-02' '2016-11-21' '2018-01-14' '2017-11-16' '2017-01-20'
 '2017-12-04' '2017-01-06' '2017-11-26' '2016-12-

In [18]:
# Split data into two sets for processing - one for home teams and another for away teams. 
# Then re-combine (stack) to compute averages statistics for games preceeding any given game.

home_data = game_data[['game_id', 
                       'season', 
                       'scheduled_date', 
                       'lead_changes', 
                       'times_tied', 
                       'periods']].copy()

away_data = home_data.copy()

for col in game_data:
    if col[0] == 'h':
        home_data.loc[:, col[2:]] = game_data[col]
        home_data.loc[:, 'at_home'] = 'True'
    elif col[0] == 'a':
        away_data.loc[:, col[2:]]= game_data[col]
        away_data.loc[:, 'at_home'] = 'False'
        
game_data_2 = home_data.append(away_data).reset_index(drop = True)
game_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19036 entries, 0 to 19035
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_id                 19036 non-null  object 
 1   season                  19036 non-null  object 
 2   scheduled_date          19036 non-null  object 
 3   lead_changes            18930 non-null  float64
 4   times_tied              18924 non-null  float64
 5   periods                 19036 non-null  float64
 6   name                    19036 non-null  object 
 7   at_home                 19036 non-null  object 
 8   alias                   19036 non-null  object 
 9   points_game             19036 non-null  int64  
 10  field_goals_made        19036 non-null  float64
 11  field_goals_att         19036 non-null  float64
 12  field_goals_pct         19036 non-null  float64
 13  three_points_made       19036 non-null  float64
 14  three_points_att        19036 non-null

In [19]:
game_data_2.head()

,game_id,season,scheduled_date,lead_changes,times_tied,periods,name,at_home,alias,points_game,...,blocks,assists_turnover_ratio,personal_fouls,foulouts,points,fast_break_pts,second_chance_pts,team_turnovers,points_off_turnovers,team_rebounds
0,8c8d2a48-2335-4acd-8304-8658b7c4f193,2017,2018-03-02,6.0,9.0,2.0,Huskies,True,WASH,79,...,4.0,1.13,15.0,0.0,79.0,5.0,6.0,0.0,14.0,2.0
1,0910b42c-d65f-44da-8076-f1bfcf2b3399,2017,2018-03-02,0.0,0.0,2.0,Sun Devils,True,ASU,84,...,7.0,1.09,19.0,0.0,84.0,6.0,10.0,0.0,21.0,5.0
2,b0cf1cc3-85c3-4a81-84a9-5c45624e0dfa,2017,2018-03-02,9.0,5.0,2.0,Cougars,True,WSU,78,...,3.0,0.75,13.0,0.0,78.0,6.0,14.0,0.0,17.0,4.0
3,4eb7e921-ecf2-4c3c-af68-590d44b9c717,2017,2018-03-02,2.0,3.0,2.0,Wildcats,True,ARIZ,75,...,6.0,1.13,13.0,0.0,75.0,0.0,12.0,0.0,6.0,3.0
4,9fd12195-25c0-4a97-8ea9-c2308b538d6b,2017,2018-03-04,10.0,3.0,2.0,Trojans,True,USC,72,...,4.0,2.20,16.0,0.0,72.0,10.0,7.0,0.0,15.0,2.0


In [20]:
# Convert 'scheduled_date' to datetime type for sorting 

game_data_2['scheduled_date'] = pd.to_datetime(game_data_2['scheduled_date'], format = '%Y-%m-%d') 

In [21]:
# Determine average stats for each team for games preceding game to be predicted.
# Calculated separately for 2016 and 2017 rosters.

averages_2016 = ncaa.calc_averages(game_data_2, '2016')
averages_2017 = ncaa.calc_averages(game_data_2, '2017')

In [24]:
# Combine 2016 and 2017 average statistics into single DataFrame. 
averages = averages_2016.append(averages_2017).sort_values(['scheduled_date'])

In [26]:
# Export processed data to file
averages.to_csv(r'../data/processed/NCAA_averages_with_index.csv', index = True, header=True) 